The Gans Project by Balu, Marvin, Frederico and Joachim 

In this project, our E-scooter company is trying to predict  effective locations in Germany to place our scooters. For the purpose of this project we are trying to attract air travellers and  tourists to use our commodity as a means of transportation. As an experiment we are limiting ourselves to 16 of the largest air hubs in Germany which are also popular tourist destinations. 

In [2]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
import re
import time
import os
import pytz
from datetime import datetime
import mysql.connector
import sqlalchemy
!pip install python-dotenv
%load_ext dotenv
%dotenv

cannot find .env file


The first task is to get the lat., long., population, country, countrycode and wiki_dataid for a list of cities in Germany and put it in a dataframe. This code is designed so that in future more cities maybe added.

In [ ]:
API_key = os.getenv("API_K_C")
#gets the API key from .env
#define the list of cities

citd =['Baden-Baden', "Berlin",  "Bonn", 'Bremen', 'Dresden', 'Dortmund', 'Düsseldorf', 'Essen', 'Frankfurt am Main', 'Hamburg', 'Hannover', 'Leipzig', "Munich", 'Münster',  'Nuremberg',  'Stuttgart']

#strategy is to first get the wikidataid and than collect the data from geoDB; openweather has no population data or at least I coul´nt see them 


API_key = os.getenv("API_K_C")
def demo(cities):
    cities_id = [] # initiate an empty id list
    dfList = []
    for city in cities:
        #retrieve the wikidataId
        time.sleep(1)
        url1 = 'https://en.wikipedia.org/wiki/{}'.format(city) #go to the wiki site of the city
        citem = requests.get(url1, 'html.parser') # get the html
        if BeautifulSoup(citem.content) != None:
            soup = BeautifulSoup(citem.content)# soup the content
        if soup.find('li', {'id':'t-wikibase'}).find('a')['href'] != None:
            wikidata_link = soup.find('li', {'id':'t-wikibase'}).find('a')['href'] #find the link that contains the wikidataid e.g. London https://www.wikidata.org/wiki/Q84
        #wl.append(wikidata_link)
        # \d+ is a regular expression and means one digit or more, the wiki data id consist of a Q followed by severaldigits
        #for group() in re see: https://www.tutorialspoint.com/What-is-the-groups-method-in-regular-expressions-in-Python
        city_id = re.search('Q\d+', wikidata_link).group()
        cities_id.append(city_id)
        #use the wikidataId to retrieve infrormation from geoDB
        url2 = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities/{}".format(city_id)
        headers = {
        "X-RapidAPI-Key": API_key,
        "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
        }    
        response = requests.request("GET", url2, headers=headers)# gets a json-like string fromg geoDB containing the necessary informations
        print(response.json())
        cit_dic = {}#make a dictionary to retrieve the information
        cit_dic['City'] = response.json()['data']['name']
        cit_dic['Country'] = response.json()['data']['country']
        cit_dic['CountryCode'] = response.json()['data']['countryCode']
        cit_dic['WikiDataId'] = response.json()['data']['wikiDataId']
        cit_dic['Latitude'] = round(response.json()['data']['latitude'], 4)
        cit_dic['Longitude'] = round(response.json()['data']['longitude'], 4)
        cit_dic['Population'] = response.json()['data']['population']
        cit_dic['Timezone'] = response.json()['data']['timezone']
        
        dfList.append(cit_dic) #put it in a list
    df_demo = pd.DataFrame(dfList) # transform the list to df
    df_demo.to_csv('./csvTables/City.csv') #writes the df to csv to the folder csv tables and replaces the table of the same name that might be already there
    return df_demo
demo(citd) #run the whole thing - good luck!

Focussing on these 15 cities, we are collecting the weather forecast. This DF is subject to change as the weather information is updated as needed.

In [ ]:
citd =['Baden-Baden', "Berlin",  "Bonn", 'Bremen', 'Dresden', 'Dortmund', 'Düsseldorf', 'Essen', 'Frankfurt am Main', 'Hamburg', 'Hannover', 'Leipzig', "Munich", 'Münster',  'Nuremberg',  'Stuttgart']#alternatively only German cities

def weather(cities):
    API_key = os.getenv("API_K_W")
    print(API_key)#gets key from the env file

    tz = pytz.timezone('Europe/Berlin')# timezone needs to be imported and translated for the european cities (Europe_Berlin to Europe/Berlin), see cell below
    now = datetime.now().astimezone(tz)#for the timestamp time of sampling

    a = {'City': [], 'Country': [], 'ForecastTime': [], 'Outlook': [], 'Temperature': [], 'Clouds': [], 'Rain': [], 'Snow': [], 'WindSpeed': [], 'DateInformation': []} #usefull form of a dic, gives the right form when change to df, see cell below 

    for city in cities: #runs a loop over the cities
        url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
        response = requests.get(url)
        json = response.json() # the first 'priciple' key for this json is 'list', which is not good practice in my op but must be used to loop over it
    #print(json['city']['name'])
    # print(type(json))

        for i in json['list']:# informations will be added to the dic a according to the json structure (see weimar cell above), if-else for rain does not work (is there or not but python cannot know that it could be there), therefore try except - appears to work but I´m not sure if clean
          #print(i)  
            a['City'].append(json['city']['name'])
            a['Country'].append(json['city']['country'])
            a['ForecastTime'].append(i['dt_txt'])
            a['Outlook'].append(i['weather'][0]['description'])
            a['Temperature'].append(i['main']['temp'])
            a['Clouds'].append(i['clouds']['all'])
            try:
                a['Rain'].append(i['rain']['3h'])
            except:
                a['Rain'].append('0')
            try:
                a['Snow'].append(i['snow']['3h'])
            except:
                a['Snow'].append('0')
            a['WindSpeed'].append(i['wind']['speed'])
            a['DateInformation'].append(now.strftime("%d/%m/%Y %H:%M:%S"))
    b=pd.DataFrame(a) #turns a to datafram , see cell below
    b.to_csv('./csvTables/Weather.csv') #writes the df to csv to the folder csv tables and replaces the table of the same name that might be already there
    return b
    
weather(citd)


In [2]:
API_key = os.getenv("API_K_W")
print(API_key)#gets key from the env file

d0040c9b033d6a3a4772784afd988afa


For the cities in our database we now get the details of the airports. 

In [ ]:
from IPython.display import JSON
#citd =['Baden-Baden', "Berlin",  "Bonn", 'Bremen', 'Dresden', 'Dortmund', 'Düsseldorf', 'Essen', 'Frankfurt am Main', 'Hamburg', 'Hannover', 'Leipzig', "Munich", 'Münster',  'Nuremberg',  'Stuttgart']
ac = ['EDSB', 'EDDT', 'EDDB', 'EDDK', 'EDDW', 'EDDC', 'EDLW', 'EDDL', 'EDDF', 'EDDH', 'EDDV', 'EDDP', 'EDDM', 'EDDG', 'EDDN', 'EDDS']

def airp(cities):
    airport = []
    API_KE = '1821798fe1mshc0aaf1be2c43cedp1289ecjsn546ec73b6483'

    for i in ac:
        url = f"https://aerodatabox.p.rapidapi.com/airports/icao/{i}"

        querystring = {"withRunways":"true"}

        headers = {
            "X-RapidAPI-Key": API_KE,
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }

        response = requests.request("GET", url, headers=headers, params=querystring)
        aa = {}#make a dictionary to retrieve the information
        aa['City'] = response.json()['municipalityName']
        aa['icao'] = response.json()['icao']
        aa['iata'] = response.json()['iata']
        aa['Time_Zone'] = response.json()['timeZone']
        aa['Country'] = response.json()['country']['name']
        aa['Country_Code'] = response.json()['country']['code']
        aa['Continent'] = response.json()['continent']['name']
        try:
            aa['Website'] = response.json()['urls']['webSite']              
        except:
            aa['Website'] = 'Not available'
        airport.append(aa)
    b = pd.DataFrame(airport)
    b.to_csv('./csvTables/Airports.csv') #writes the df to csv to the folder csv tables and replaces the table of the same name that might be already there
    return b 

airp(ac)


For all the airports in our list we can have the  details of the arriving flights. The most important detail being the flight times. The list may also optionally include the details of the flights origin as in future we can have a statistical analysis of the passengers who may use our scooters. This is also a changing DF and with queries we can find out the most and least busiest times of the day. This code is not ideal since it does not show the destination airport. Need to edit the codes.

In [ ]:
import os

API_KE = 'd67656b1a1mshbd01e446e3b088dp1b216ejsn1abb69340012' #os.getenv("API_C_F")
#print(API_KE)
cit = ['EDSB', 'EDDT', 'EDDB', 'EDDK', 'EDDW', 'EDDC', 'EDLW', 'EDDL', 'EDDF', 'EDDH', 'EDDV', 'EDDP', 'EDDM', 'EDDG', 'EDDN', 'EDDS']
def flights(cities):
    
    a = {'City': [], 'FlightNumber': [], 'FlightStatus': [], 'OriginAirport': [], 'ScheduledTime': [], 'AirLine': []}#, 'Outlook': [], 'Temperature': [], 'Clouds': [], 'Rain': [], 'Snow': [], 'WindSpeed': [], 'DateInformation': []}
    for city in cities:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{city}/2022-06-15T06:00/2022-06-15T17:00"
        querystring = {"direction":"Arrival","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"true"}
        headers = {
           "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
           "X-RapidAPI-Key": API_KE
           }
        responses = requests.request("GET", url, headers=headers, params=querystring)
        json = responses.json()
        print(json)
        
        for i in json['arrivals']:
            a['City'] = city
            a['FlightNumber'].append(i['number'])
            a['FlightStatus'].append(i['status'])
            a['OriginAirport'].append(i['movement']['airport']['name'])
            a['ScheduledTime'].append(i['movement']['scheduledTimeLocal'])
            a['AirLine'].append(i['airline']['name'])
        
    c = pd.DataFrame(a)
    c.to_csv('./csvTables/Flights.csv') #writes the df to csv to the folder csv tables and replaces the table of the same name that might be already there
    return c
flights(cit)



{'arrivals': [{'movement': {'airport': {'icao': 'LEPA', 'iata': 'PMI', 'name': 'Palma De Mallorca'}, 'scheduledTimeLocal': '2022-06-15 10:25+02:00', 'scheduledTimeUtc': '2022-06-15 08:25Z', 'quality': ['Basic']}, 'number': 'EW 6804', 'callSign': 'EWG67R', 'status': 'Unknown', 'codeshareStatus': 'IsOperator', 'isCargo': False, 'aircraft': {'reg': 'D-ABFO', 'modeS': '3C48CF', 'model': 'Airbus A320'}, 'airline': {'name': 'Eurowings'}}, {'movement': {'airport': {'icao': 'LEPA', 'iata': 'PMI', 'name': 'Palma De Mallorca'}, 'scheduledTimeLocal': '2022-06-15 11:30+02:00', 'scheduledTimeUtc': '2022-06-15 09:30Z', 'quality': ['Basic']}, 'number': 'FR 5183', 'status': 'Unknown', 'codeshareStatus': 'Unknown', 'isCargo': False, 'aircraft': {'model': 'Boeing 737-800'}, 'airline': {'name': 'Ryanair'}}, {'movement': {'airport': {'icao': 'LATI', 'iata': 'TIA', 'name': 'Tirana'}, 'scheduledTimeLocal': '2022-06-15 13:30+02:00', 'scheduledTimeUtc': '2022-06-15 11:30Z', 'quality': ['Basic']}, 'number': 'W

In [ ]:
demo(citd)

In [ ]:
weather(citd)

In [ ]:
flights(cit)

In [ ]:
airp(ac)

After completing all the DFs, they are exported to an already created SQL Schema. Note: No tables were created in MySQL workbench all tables are created from here. Once created go to sql to create Primary and Foreign Keys. Also the flights DF is not exported as the city column is missing and I have ran out of API calls.

In [50]:

# define connection details
cnx = mysql.connector.connect(
    user='root',
    password='Majupau7##1.0', #type your root password here
    host='127.0.0.1', # to connect to your local instance
    database='scooter' #type the name of the database you want to use here
)
 
# connect to database
cursor   = cnx.cursor()
schema   ="scooter"
host     ="127.0.0.1"
user     ="root"
password = pw
port     = 3306
con      = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
cnx = mysql.connector.connect(
    user='admin',
    password='!Kochumol20', #type your root password here
    host='gans.czi7af4kh5fl.us-east-1.rds.amazonaws.com', # to connect to your local instance
    database='gans' #type the name of the database you want to use here
)
 
# connect to database
cursor = cnx.cursor()
schema="gans"
host="gans.czi7af4kh5fl.us-east-1.rds.amazonaws.com"
user="admin"
password="!Kochumol20"
port=3306
on = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
# !pip install pymysql
# weather(citd).to_sql('weather', 
#               if_exists='append', 
#               con=con, 
#               index=False)
# airp(ac).to_sql('airports', 
#               if_exists='append', 
#               con=con, 
#               index=False)
# demo(citd).to_sql('cities', 
#               if_exists='append', 
#               con=con, 
#               index=False)
# flights(cit).to_sql('flights', 
#               if_exists='append', 
#               con=con, 
#               index=False)